In [1]:
import Pkg;

In [2]:
using Random
using Images

In [3]:
main_directory = "output"

"output"

In [28]:
function extract_base_name(filename::String)::String
  pattern = r"(.+?)(?:_predict_denoised)?\.png$"
  match_result = match(pattern, filename)
  if match_result !== nothing
      return match_result.captures[1]
  else
      return "Invalid filename format."
  end
end

function obtain_png_name(base_name::String, pred_denoise::Bool=false)::String
  if pred_denoise
      return base_name * "_predict_denoised.png"
  else
      return base_name * ".png"
  end
end

function dice_loss(y_true::Array{Float32,2}, y_pred::Array{Float32,2})::Float32
  if size(y_true) != size(y_pred)
      error("Input images must have the same dimensions")
  end
  y_true_flat = vec(y_true)
  y_pred_flat = vec(y_pred)
  intersection = sum(y_true_flat .* y_pred_flat)
  true_sum = sum(y_true_flat)
  pred_sum = sum(y_pred_flat)
  dice_coefficient = 2 * intersection / (true_sum + pred_sum)
  return 1.0 - dice_coefficient
end

function image_dice_loss(path1::String, path2::String)
  image1 = load(path1)
  image2 = load(path2)
  image1_binary = Float32.(Gray.(image1) .> 0.5)
  image2_binary = Float32.(Gray.(image2) .> 0.5)
  loss = dice_loss(image1_binary, image2_binary)
  return loss
end

"""Returns tuples of (output/.../fati/1234.png, output/.../sohrab/1234.png, output/.../predict/1234_predict_denoised.png)."""
function get_path_tuples(subdirectory::String)
  fati_dir = joinpath(subdirectory, "fati")
  sohrab_dir = joinpath(subdirectory, "sohrab")
  predict_dir = joinpath(subdirectory, "predict")

  fati_list, sohrab_list, predict_list = [], [], []

  for file in readdir(fati_dir, join=true)
    if occursin(r"\.png$", file)
      push!(fati_list, basename(file))
    end
  end

  for file in readdir(sohrab_dir, join=true)
    if occursin(r"\.png$", file)
      push!(sohrab_list, basename(file))
    end
  end

  for file in readdir(predict_dir, join=true)
    if occursin(r"\.png$", file)
      push!(predict_list, basename(file))
    end
  end

  overlap_names = intersect(extract_base_name.(fati_list), extract_base_name.(sohrab_list), extract_base_name.(predict_list))
  all_path_tuples = []

  for name in overlap_names
    fati_path = joinpath(fati_dir, obtain_png_name(name))
    sohrab_path = joinpath(sohrab_dir, obtain_png_name(name))
    predict_path = joinpath(predict_dir, obtain_png_name(name, true))

    path_tuple = [fati_path, sohrab_path, predict_path]
    push!(all_path_tuples, path_tuple)
  end
  return all_path_tuples
end

comparable_paths (generic function with 1 method)

In [33]:
for subdirectory in readdir(main_directory, join=true)
  if isdir(subdirectory) && occursin(r"^SID", basename(subdirectory))
    all_path_tuples = get_path_tuples(subdirectory)
    for (fati_path, sohrab_path, predict_path) in all_path_tuples
      loss1, loss2, loss3 = image_dice_loss(fati_path, sohrab_path), image_dice_loss(fati_path, predict_path), image_dice_loss(sohrab_path, predict_path)
    end
  end
end

0.16841489
0.46768725
0.40190053
0.18760884
0.43839622
0.49056995
0.16307414
0.3230551
0.38510793
0.12711692
0.46670526
0.43651295
